In [2]:
import numpy as np
import os 

In [3]:
dirname = './02-04/'
a0 = -45
a1 = 0 
b0 = -22.5 
b1 = 22.5

angles = [
    (a0, b0),
    (a0 + 90, b0 + 90),
    (a0 + 90, b0),
    (a0, b0 + 90),
    (a0, b1),
    (a0 + 90, b1 + 90),
    (a0 + 90, b1),
    (a0, b1 + 90),
    (a1, b0),
    (a1 + 90, b0 + 90),
    (a1 + 90, b0),
    (a1, b0 + 90),
    (a1, b1),
    (a1 + 90, b1 + 90),
    (a1 + 90, b1),
    (a1, b1 + 90),
]

coinc_window = 40e-9
acq_time = 1

In [7]:
n = {} 
for ang in angles: 
    n[ang] = np.mean(np.loadtxt(os.path.join(dirname, f'{ang[0]}_{ang[1]}.txt')), axis=0)[-1]

nb = {} 
for ang in angles: 
    nb[ang] = np.mean(np.loadtxt(os.path.join(dirname, f'{ang[0]}_{ang[1]}.txt')), axis=0)[1]

nbp = {} 
for ang in angles: 
    nbp[ang] = np.mean(np.loadtxt(os.path.join(dirname, f'{ang[0]}_{ang[1]}.txt')), axis=0)[3]



In [8]:
for a in angles: 
    row = f'${a[0]}^\\circ$ & ${a[1]}^\\circ$ & ' + \
            f'{round(nb[a])} & {round(nbp[a])} & {round(n[a])} & ' + \
            f'{round(nb[a] * nbp[a] * coinc_window / acq_time)} \\\\'
    print(row)

$-45^\circ$ & $-22.5^\circ$ & 25973 & 20043 & 729 & 21 \\
$45^\circ$ & $67.5^\circ$ & 26478 & 22288 & 763 & 24 \\
$45^\circ$ & $-22.5^\circ$ & 25585 & 20316 & 226 & 21 \\
$-45^\circ$ & $67.5^\circ$ & 26184 & 21735 & 261 & 23 \\
$-45^\circ$ & $22.5^\circ$ & 26188 & 19693 & 196 & 21 \\
$45^\circ$ & $112.5^\circ$ & 25490 & 22602 & 288 & 23 \\
$45^\circ$ & $22.5^\circ$ & 26013 & 20004 & 670 & 21 \\
$-45^\circ$ & $112.5^\circ$ & 26440 & 22404 & 804 & 24 \\
$0^\circ$ & $-22.5^\circ$ & 24332 & 20391 & 774 & 20 \\
$90^\circ$ & $67.5^\circ$ & 28611 & 22215 & 860 & 25 \\
$90^\circ$ & $-22.5^\circ$ & 28959 & 20673 & 193 & 24 \\
$0^\circ$ & $67.5^\circ$ & 24229 & 21979 & 196 & 21 \\
$0^\circ$ & $22.5^\circ$ & 24371 & 19877 & 776 & 19 \\
$90^\circ$ & $112.5^\circ$ & 27877 & 22576 & 915 & 25 \\
$90^\circ$ & $22.5^\circ$ & 28550 & 19846 & 132 & 23 \\
$0^\circ$ & $112.5^\circ$ & 24190 & 22841 & 109 & 22 \\


## Calculate S

In [9]:
def E(alpha, beta): 
    a_b, ap_b, a_bp, ap_bp = [n[(alpha, beta)], n[(alpha + 90, beta)], n[(alpha, beta + 90)], n[(alpha + 90, beta + 90)]]
    return (a_b + ap_bp - a_bp - ap_b) / (a_b + ap_bp + a_bp + ap_b)

S = E(a0, b0) - E(a0, b1) + E(a1, b0) + E(a1, b1)

## Error propagation

In [10]:
ps = [(0, 0), (90, 90), (90, 0), (0, 90)]
dss = {}


for a, b in [(a0, b0), (a1, b0), (a0, b1), (a1, b1)]:
    ntot = 0 

    for pa, pb in ps:
        ntot += n[(a + pa, b + pb)]

    for pa, pb in ps: 
        factor = 1
        if pa + pb == 90: 
            factor = -1
    
        dss[(a + pa, b + pb)] = (factor * ntot - 1) / ntot**2 

std = np.sqrt(sum(n[(a, b)] * dss[(a, b)]**2 for a, b in angles))
print(f'S = {'{:.4}'.format(S)} ± {'{:.3}'.format(std)}') 

S = 2.378 ± 0.045
